In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time

id = input("ID : ")
pwd = input("PWD : ")

driver=webdriver.Chrome(executable_path="chromedriver")
driver.set_window_size(1400,1000)
driver.get("https://cse.cau.ac.kr/main.php")
driver.find_element(By.XPATH,'//*[@id="top"]/div[2]/div/div[1]/a[5]').click()
driver.find_element(By.XPATH,'//*[@id="sub05"]/div/a[4]').click()



In [2]:
driver.find_element(By.XPATH,'//*[@id="txtUserID"]').send_keys(id)
driver.find_element(By.XPATH,'//*[@id="txtPwd"]').send_keys(pwd, Keys.ENTER)

time.sleep(1)
driver.switch_to.alert.accept()

In [26]:
# driver.find_element(By.XPATH, '//*[@id="content"]/form/table[1]/tbody/tr/td/a').click() #달력 들어가기
# tbody = driver.find_element(By.XPATH, '//*[@id="content"]/table[3]/tbody')
# for tr in tbody.find_elements(By.TAG_NAME, 'tr'):
#     for td in tr.find_elements(By.TAG_NAME, 'td'):
#         print(td.get_attribute("innerText"))

In [33]:
choice = 0
while choice != 1:
    mon = input("mon : ")
    day = input("day : ")
    num = int(input("num : "))

    Select(driver.find_element(By.XPATH,'//*[@id="reserveMonth"]')).select_by_value(str(mon).zfill(2))
    Select(driver.find_element(By.XPATH,'//*[@id="reserveDay"]')).select_by_value(str(day).zfill(2))
    Select(driver.find_element(By.XPATH,'//*[@id="classRoomNo"]')).select_by_index(num)

    list_tbody = driver.find_element(By.XPATH, '//*[@id="content"]/table/tbody')
    for tr in list_tbody.find_elements(By.TAG_NAME, 'tr'):
        print(tr.get_attribute("innerText"))
    
    choice = int(input("1. 예약하기\n2. 다른날짜"))



팀프로젝트실5 (208관 601호)	2023-06-01	17:00 ~ 18:00	관리*(admin)	2	수업	-
예약정보가 없습니다.


In [18]:
stt_hour = int(input("hour : "))
stt_min = int(input("1. 정각  2. 30분"))-1
time_select = int(input("1. 30분  2. 1시간  3. 1시간 30분  4. 2시간"))
end_hour = stt_hour + (stt_min+time_select)//2
end_min = (stt_min+time_select)%2
person = int(input("인원수 : "))
purpose = input("이용 목적 : ")

Select(driver.find_element(By.XPATH, '//*[@id="reserveStartTimeH"]')).select_by_value(str(stt_hour).zfill(2))
Select(driver.find_element(By.XPATH, '//*[@id="reserveStartTimeM"]')).select_by_index(stt_min)
Select(driver.find_element(By.XPATH, '//*[@id="reserveEndTimeH"]')).select_by_value(str(end_hour).zfill(2))
Select(driver.find_element(By.XPATH, '//*[@id="reserveEndTimeM"]')).select_by_index(end_min)
Select(driver.find_element(By.XPATH, '//*[@id="reserveMember"]')).select_by_index(person-1)
driver.find_element(By.XPATH, '//*[@id="memo"]').send_keys(purpose)